In [1]:
import os
from datetime import datetime, date, timedelta
import pandas as pd 
import numpy as np


In [ ]:

folder_path = 'D:/Nifty_1second'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
sorted_files = sorted(csv_files, key=lambda x: datetime.strptime(x.split('_')[0] + '_' + x.split('_')[1] + '_' + x.split('_')[2], '%Y_%m_%d'))

for i in range(0, len(sorted_files)):
    print(sorted_files[i])

    file_path = os.path.join(folder_path, sorted_files[i])
    df = pd.read_csv(file_path)

    # --------------------------------------

    time_range = pd.date_range(start='09:15:00', end='15:30:00', freq='5S').strftime('%H:%M:%S')

    # Convert datetime to time string for comparison
    df['time'] = pd.to_datetime(df['datetime']).dt.strftime('%H:%M:%S')

    # Get unique symbols
    symbols = df['symbol'].unique()

    missing_rows = []
    new_rows = []

    current_date = str(pd.to_datetime(df['datetime'].iloc[0]).date())

    for symbol in symbols:
        symbol_df = df[df['symbol'] == symbol]
        
        # Find missing times for this symbol
        symbol_times = symbol_df['time'].values
        missing_times = set(time_range) - set(symbol_times)
        
        if len(missing_times) > 0:
            # print(f"Missing rows for {symbol}:")
            for missing_time in missing_times:
                # print(f"Time: {missing_time}")
                
                # Find nearest previous row
                prev_rows = symbol_df[symbol_df['time'] < missing_time]
                next_rows = symbol_df[symbol_df['time'] > missing_time]
                
                # If no previous row found, try to get next row
                if len(prev_rows) == 0:
                    if len(next_rows) == 0:
                        continue  # Skip if neither previous nor next rows found
                    # Use next row's open price
                    next_time = next_rows.iloc[0]
                    new_row = next_time.copy()
                    new_row['datetime'] = current_date + " " + missing_time
                    new_row['time'] = missing_time
                    new_row['open'] = next_time['open']
                    new_row['high'] = next_time['open']
                    new_row['low'] = next_time['open']
                    new_row['close'] = next_time['open']
                else:
                    # Use previous row's close price
                    prev_time = prev_rows.iloc[-1]
                    new_row = prev_time.copy()
                    new_row['datetime'] = current_date + " " + missing_time
                    new_row['time'] = missing_time
                    new_row['open'] = prev_time['close']
                    new_row['high'] = prev_time['close']
                    new_row['low'] = prev_time['close']
                    new_row['close'] = prev_time['close']

                new_row['volume'] = 0

                new_rows.append(new_row)

    # Add new rows to dataframe
    if new_rows:
        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
        
    # Sort by datetime and symbol
    df = df.sort_values(['symbol', 'datetime'])

    # Drop temporary time column
    df = df.drop('time', axis=1)

    df.to_csv(file_path, index=False)
